The University of Tokyo

Special Lectures in Information Science Ⅱ  

Introduction to Near-Term Quantum Computing  

# 4. Quantum Algorithms: Phase estimation

Yoshiaki Kawase (May 02, 2025)  

*Approximate QPU time to run this experiment is 13 seconds.*

This notebook provides the fundamental concepts and implementation of the Quantum Fourier Transformation (QFT) and Quantum Phase Estimation (QPE). 

**Quantum Fourier Transformation (QFT)**

The Quantum Fourier Transformation is the quantum counterpart of the classical discrete Fourier transform. It is a linear transformation applied to the quantum states, mapping computational basis into their Fourier basis representations. The QFT plays a key role in many quantum algorithms, offering an efficient methods to extract periodicity information from quantum states.
The QFT can be implemented with $O(N^2)$ with quantum gates such as Hadamard gates and Control-Phase gates for $N$ qubits, enabling exponential speedup over classical Fourier transformation.

- **Applications**: It is a foundational part in quantum algorithms such as Shor's algorithm for factoring large integers and discrete logarithm.


**Quantum Phase Estimation (QPE)**

Quantum Phase Estimation is a quantum algorithm used to estimate the phase associated with an eigenvector of a unitary operator. This algorithm provides a bridge between the abstract mathematical properties of quantum states and their computational applications.


- **Applications**: It can solve problems such as finding eigenvalues of unitary matrices and simulating quantum systems.


Together, QFT and QPE form the essential backbone of many quantum algorithms solving problems that are infeasible for classical computers. By the end of this notebook, you will gain an understanding of how these techniques are implemented.

In [ ]:
# Required packages for Google Colab
#%pip install qiskit[visualization]
#%pip install qiskit-aer
#%pip install qiskit-ibm-runtime

In [ ]:
import qiskit
qiskit.__version__

## Basic of Quantum Fourier Transformation (QFT)

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import *
from qiskit.quantum_info import Statevector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Sampler

from numpy import pi

From the analogy with the discrete fourier transform, the QFT acts on a quantum state $\vert X\rangle = \sum_{j=0}^{N-1} x_j \vert j \rangle$ for $N$ qubits and maps it to the quantum state $\vert Y\rangle = \sum_{k=0}^{N-1} y_k \vert k \rangle$.


$$QFT_{N}: \vert j \rangle \mapsto \frac{1}{\sqrt{N}}\sum_{k=0}^{N-1}\omega_N^{jk} \vert k \rangle$$

where $\omega_N^{jk} = e^{2\pi i \frac{jk}{N}}$.


Or the unitary matrix representation:


$$ U_{QFT} = \frac{1}{\sqrt{N}} \sum_{j=0}^{N-1} \sum_{k=0}^{N-1} \omega_N^{jk} \vert k \rangle \langle j \vert$$

### Example: 1-qubit QFT

Let's consider the case of $N = 2^1 = 2$.

The unitary matrix can be written:

$$ 
\begin{aligned}
U_{QFT} & = \frac{1}{\sqrt{2}} \sum_{j=0}^{1} \sum_{k=0}^{1} \omega_2^{jk} \vert k \rangle \langle j \vert
\\
& = \frac{1}{\sqrt{2}} (\omega_2^{0} \vert 0 \rangle \langle 0 \vert + \omega_2^{0} \vert 0 \rangle \langle 1 \vert + \omega_2^{0} \vert 1 \rangle \langle 0 \vert + \omega_2^{1} \vert 1 \rangle \langle 1 \vert)
\\
& = \frac{1}{\sqrt{2}} (\vert 0 \rangle \langle 0 \vert + \vert 0 \rangle \langle 1 \vert + \vert 1 \rangle \langle 0 \vert - \vert 1 \rangle \langle 1 \vert)
\\
& = H
\end{aligned}
$$

This operation is the result of applying the Hadamard gate($H$).

### Product representation of QFT

Let's generalize a transformation for $N = 2^n$, $QFT_{N}$ acting on the state $\vert x \rangle = \vert x_1\ldots x_n \rangle$.
$$
\begin{aligned}
QFT_N\vert x \rangle & = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1}\omega_N^{xy} \vert y \rangle
\\
& = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} e^{2 \pi i xy / 2^n} \vert y \rangle ~\text{since}\: \omega_N^{xy} = e^{2\pi i \frac{xy}{N}} \:\text{and}\: N = 2^n
\\
& = \frac{1}{\sqrt{N}} \sum_{y=0}^{N-1} e^{2 \pi i \left(\sum_{k=1}^n y_k/2^k\right) x} \vert y_1 \ldots y_n \rangle \:\text{rewriting in fractional binary notation}\: y = y_1\ldots y_n, y/2^n = \sum_{k=1}^n y_k/2^k
\\
& = \frac{1}{\sqrt{N}} \sum_{y_1=0}^{1} \cdots \sum_{y_n=0}^{1} \prod_{k=1}^n e^{2 \pi i x y_k/2^k } \vert y_1 \ldots y_n \rangle \:\text{after expanding the exponential of a sum to a product of exponentials and expanding}
\sum_{y=0}^{N-1} = \sum_{y_1=0}^{1}\sum_{y_2=0}^{1}\ldots\sum_{y_n=0}^{1}
\\
& = \frac{1}{\sqrt{N}} \bigotimes_{k=1}^n  \left(\vert0\rangle + e^{2 \pi i x /2^k } \vert1\rangle \right) \:\text{after rearranging the sum and products}
\\
& = \frac{1}{\sqrt{N}}
\left(\vert0\rangle + e^{\frac{2\pi i}{2}x} \vert1\rangle\right)
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^2}x} \vert1\rangle\right)
\otimes  
\ldots
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^{n-1}}x} \vert1\rangle\right)
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^n}x} \vert1\rangle\right)
\end{aligned}
$$

### Example: Circuit construction of 3 qubits QFT

In [ ]:
# Prepare for 3 qubits circuit
qr = QuantumRegister(3) 
cr = ClassicalRegister(3)
qc = QuantumCircuit(qr,cr)

In [ ]:
qc.h(2)
qc.cp(pi/2, 1, 2) # Controlled-phase gate from qubit 1 to qubit 2
qc.cp(pi/4, 0, 2) # Controlled-phase gate from qubit 0 to qubit 2
qc.draw(output="mpl")

In [ ]:
qc.h(1)
qc.cp(pi/2, 0, 1) # Controlled-phase gate from qubit 0 to qubit 1

qc.draw(output="mpl")

In [ ]:
qc.h(0)

qc.draw(output="mpl")

In [ ]:
qc.swap(0,2)

qc.draw(output="mpl")

We try to apply QFT to $|5\rangle$ as an example.

First, we confirm the binary notation of the integer 5 and create the circuit encoding the state 5:

In [ ]:
bin(5)

In [ ]:
qc = QuantumCircuit(3)

qc.x(0)
qc.x(2)
qc.draw(output="mpl")

We check the quantum states using the Aer simulator:

In [ ]:
statevector = Statevector(qc)
plot_bloch_multivector(statevector)

Finally, we add QFT and view the final state of our qubits:

In [ ]:
qc.h(2)
qc.cp(pi/2, 1, 2)
qc.cp(pi/4, 0, 2)

qc.h(1)
qc.cp(pi/2, 0, 1)

qc.h(0)

qc.swap(0,2)

qc.draw(output="mpl")

In [ ]:
statevector = Statevector(qc)
plot_bloch_multivector(statevector)

We can see out QFT function has worked correctly. Qubit 0 has been rotated by $\tfrac{5}{8}$ of a full turn, qubit 1 by $\tfrac{10}{8}$ full turns (equivalent to $\tfrac{1}{4}$ of a full turn), and qubit 2 by $\tfrac{20}{8}$ full turns (equivalent to $\tfrac{1}{2}$ of a full turn).

$\because QFT_8\vert 5 \rangle = \frac{1}{\sqrt{8}}
\left(\vert0\rangle + e^{\frac{2\pi i}{2}5} \vert1\rangle\right)
\otimes
\left(\vert0\rangle + e^{\frac{2\pi i}{2^2}5} \vert1\rangle\right)
\otimes  
\left(\vert0\rangle + e^{\frac{2\pi i}{2^3}5} \vert1\rangle\right)$

## Exercise: QFT

(1) Implement QFT of 4 qubits. 

In [ ]:
##your code goes here##







(2) Apply QFT to $|14\rangle$, simulate and plot the statevector using the bloch sphere.

In [ ]:
##your code goes here##

# prepare |14>, using X gates

# apply QFT to |14>



## Basic of Quantum Phase Estimation (QPE)

Given a unitary operation $U$, the QPE estimates $\theta$ in $U\vert\psi \rangle =e^{\boldsymbol{2\pi i} \theta }|\psi \rangle$ since $U$ is unitary, all of its eigenvalues have a norm of 1.

### Procedure

#### 1. Setup
$\vert\psi\rangle$ is in one set of qubit registers. An additional set of $n$ qubits form the counting register on which we will store the value $2^n\theta$: 

$$ |\psi_0\rangle = \lvert 0 \rangle^{\otimes n} \lvert \psi \rangle$$

#### 2. Superposition

Apply a $n$-bit Hadamard gate operation $H^{\otimes n}$ on the counting register: 



$$ |\psi_1\rangle = {\frac {1}{2^{\frac {n}{2}}}}\left(|0\rangle +|1\rangle \right)^{\otimes n} \lvert \psi \rangle$$

#### 3. Controlled Unitary operations

We need to introduce the controlled unitary $CU$ that applies the unitary operator $U$ on the target register only if its corresponding control bit is $|1\rangle$. Since $U$ is a unitary operator with eigenvector $|\psi\rangle$ such that $U|\psi \rangle =e^{\boldsymbol{2\pi i} \theta }|\psi \rangle$, this means: 



$$U^{2^{j}}|\psi \rangle =U^{2^{j}-1}U|\psi \rangle =U^{2^{j}-1}e^{2\pi i\theta }|\psi \rangle =\cdots =e^{2\pi i2^{j}\theta }|\psi \rangle$$

### Example: T-gate QPE

Let's use $T$ gate as an example of QPE and estimate its phase $\theta$.

$$ T|1\rangle = 
\begin{bmatrix}
1 & 0\\
0 & e^\frac{i\pi}{4}\\ 
\end{bmatrix}
\begin{bmatrix}
0\\
1\\ 
\end{bmatrix}
= e^\frac{i\pi}{4}|1\rangle $$

We expect to find:

$$\theta = \frac{1}{8}$$

where

$$ T|1\rangle = e^{2i\pi\theta}|1\rangle $$

We initialize $\vert\psi\rangle = \vert1\rangle$ of the eigenvector of $T$ gate by applying an $X$ gate:

In [ ]:
qpe = QuantumCircuit(4, 3)
qpe.x(3)
qpe.draw(output='mpl')

Next, we apply Hadamard gates to the counting qubits:

In [ ]:
for qubit in range(3):
    qpe.h(qubit)
qpe.draw(output='mpl')

We perform the controlled unitary operations:

In [ ]:
repetitions = 1
for counting_qubit in range(3):
    for i in range(repetitions):
        qpe.cp(pi/4, counting_qubit, 3); # This is C-U
    repetitions *= 2
qpe.draw(output='mpl')

We apply the inverse quantum Fourier transformation to convert the state of the counting register, then measure the counting register:

In [ ]:
from qiskit.circuit.library import QFT

In [ ]:
# Apply inverse QFT
qpe.append(QFT(3, inverse=True), [0,1,2])
qpe.draw(output='mpl')

In [ ]:
for n in range(3):
    qpe.measure(n,n)
qpe.draw(output='mpl')

We simulate using Aer simulator:

In [ ]:
aer_sim = AerSimulator()
shots = 2048

pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=1)
t_qpe = pm.run(qpe)

sampler = Sampler(mode=aer_sim)
job = sampler.run([t_qpe], shots=shots)
result = job.result()
answer = result[0].data.c.get_counts()

plot_histogram(answer)

We see we get one result (`001`) with certainty, which translates to the decimal: `1`. We now need to divide our result (`1`) by $2^n$ to get $\theta$:



$$ \theta = \frac{1}{2^3} = \frac{1}{8} $$



 Shor's algorithm allows us to factorize a number by building a circuit with $\theta$ unknown and obtaining $\theta$.

## Exercise: $\theta = 1/3$

Estimate $\theta = 1/3$ using 3 qubits for counting and a qubit for an eigenvector.

$P|1\rangle = e^{i\lambda}|1\rangle$. Since we want to implement $U|1\rangle = e^{2\pi i \tfrac{1}{3}}|1\rangle$, we need to set $\lambda = \tfrac{2 \pi}{3}$.

In [ ]:
##your code goes here##
# Create and set up circuit

# Apply H-Gates to counting qubits:

# Prepare our eigenstate |1>:

# apply the controlled-U operations:

# apply the inverse QFT:

# measurement



## Execution using Qiskit Runtime Primitives Sampler

We will perform QPE using the real quantum device and follow 4 steps of Qiskit Patterns.

    1. Map the problem to a quantum-native format
    2. Optimize the circuits
    3. Execute the target circuit
    4. Postprocess the results

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Sampler

In [ ]:
# for Google Colab users
#service = QiskitRuntimeService(
#    channel='ibm_quantum',
#    instance='ibm-q/open/main',
#    token='<IBM Quantum API key>')

#service.backends()

In [ ]:
# for users who installed Qiskit in your local environment
# If this is your first time running your circuit on a real device
# QiskitRuntimeService.save_account(channel='ibm_quantum', instance='ibm-q/open/main', token='<IBM Quantum API key>', overwrite=True)
service = QiskitRuntimeService()
service.backends()

In [ ]:
#real_backend = service.backend("ibm_kyiv")
#real_backend = service.backend("ibm_kawasaki")

In [ ]:
#You can also identify the least busy device

real_backend = service.least_busy(simulator=False, operational=True)
print("The least busy device is ", real_backend)

#### Step 1. Map the problem to a quantum-native format

In [ ]:
# Create and set up circuit
qpe = QuantumCircuit(4, 3)

# Apply H-Gates to counting qubits:
for qubit in range(3):
    qpe.h(qubit)

# Prepare our eigenstate |psi>:
qpe.x(3)

# Do the controlled-U operations:
angle = 2*pi/3
repetitions = 1
for counting_qubit in range(3):
    for i in range(repetitions):
        qpe.cp(angle, counting_qubit, 3)
    repetitions *= 2

# Do the inverse QFT:
qpe.append(QFT(3,inverse=True), [0,1,2])

for n in range(3):
    qpe.measure(n,n)

qpe.draw(output='mpl')

#### Step 2. Optimize the circuits

In [ ]:
# Transpile the circuit into basis gates executable on the hardware
pm = generate_preset_pass_manager(backend=backend, optimization_level=2)
qc_compiled = pm.run(qpe)

qc_compiled.draw("mpl", idle_wires=False)

#### Step 3. Execute the target circuit

In [ ]:
real_sampler = Sampler(mode=backend)
job = real_sampler.run([qc_compiled], shots=1024)
print("job id:", job.job_id())

In [ ]:
job = service.job('cxb5pzbrkac00089r7v0') # Input your job-id between the quotations
job.status()

In [ ]:
result_real = job.result()
print(result_real)

#### Step 4. Postprocess the results

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(result_real[0].data.c.get_counts())